# Equilibrium neutron capture nucleosynthesis

#### Where
`/user/scratch14_wendi3/dpa/NuPPN/examples/ppn_nconst-xi`

#### Author
Falk Herwig, Feb 15, 2019


## Goal
The goal is to prescribe a method of configuring single-zone simulations that generate an equilibrium trans-Fe element abundance pattern for a prescribed, constant neutron density. With this method one can determine local elemental abundance ratios as a function of neutron density once equilibrium is reached, independent of neutron exposure.

## Preliminaries

The network equations are written in terms of the number density $N_m$ of
species $m$ by collecting all production and destruction terms of
reactions of the type $k + l \rightarrow m +n$
\begin{eqnarray}
  \label{eqn:react}
\frac{dN_m}{dt} = N_k N_l<\sigma v>_{kl,m} - N_m N_n <\sigma v>_{mn,o} + ... +
N_i\lambda_{i,m} - N_m\lambda_{m,j}
\end{eqnarray}
where $<\sigma v>$ is the product of the cross section and the
relative velocity in the center-of-mass system averaged over the
appropriate distribution function and $\lambda$ is the rate for
$\beta$ decays. 

The number density is expressed in terms of a number fraction or mole
fraction $Y=X/A$, with $A$ the atomic mass number, by $N=Y \rho                                          
N_\mathrm{A}$, where $N_\mathrm{A} =1/M_\mathrm{u}$ is the Avogadro number, and
$M_\mathrm{u}$ is the atomic mass number. 

## Constant neutron density in the network equations
$\mathsf{N_n}$ is the neutron density. To have constant neutron density implies the following (quasi-)equilibrium version of the rate equation:

$$
\mathsf{\frac{dN_\mathrm{n}}{dt} = 0 = N_k N_l<\sigma v>_{kl,n} - \sum_{m = 1}^{M} N_m N_n <\sigma v>_{mn,o}},
$$

where $\mathsf{N_k}$  and  $\mathsf{N_l}$  are the reactants of the neutron source reaction. It is sufficient to have only one neutron source reaction since we are not modeling the neutron source realistically anyways. $\mathsf{N_m}$  are neutron capturing species, such as  $\mathsf{^{56}Fe}$ and all the other trans-Fe isotopes up to $\mathsf{^{209}Bi}$. 

## General idea
The idea has two parts. One is how to formulate the neutron source as a reaction that ensures a prescribed neutron density. The other involves a reaction from $\mathsf{^{209}Bi}$ back to $\mathsf{^{56}Fe}$ to ensure the former is not piling up and the latter is not depleted in the drive toward an equilibrium solution.

## Neutron source
The above equilibrium equation can be used to calculate the reaction rate  $\mathsf{<\sigma v>_{kl,n}}$ of the  neutron source as all other variables are known, including the prescribed neutron density  $\mathsf{N_n}$. While the composition of the neutron absorbing species $N_\mathrm{m}$ will vary a bit during the time step, and therefore $\mathsf{<\sigma v>_{kl,n}}$ should vary, this effect will vanish as equilibrium is approached. The same is true for $\mathsf{<\sigma v>_{kl,n}}$. 

Using the neutron source reaction rate $\mathsf{<\sigma v>_{kl,n}}$ as described above, we introduce an artificial neutron source reaction. We adopt two artificial species already defined in the `ppn_physics.input` sandbox, $\mathsf{^1G}$ and $\mathsf{^1L}$. This artificial neutron source is similar but just the reverse of the neutron sink introduced by [Herwig+ 03](http://adsabs.harvard.edu/abs/2003ApJ...593.1056H).

```
    G1 + L1 -> n + G1
```

One may think of $\mathsf{^1L}$ as the artificial equivalent of $\mathsf{^4He}$, and $\mathsf{^1G}$ as the equivalent of $\mathsf{^{13}C}$. 

### Network for trans-Fe elements only
Here we are only interested in trans-iron nucleosynthesis. All abundances of species lighter than Fe will be set to zero (see below). This means that $\mathsf{^{56}Fe}$ is the lightest neutron capture element and no leakage from lighter elements into the trans-Fe region is allowed. This is equivalent to saying that there is an unlimited (or very large) reservoir of Fe seeds, which is indeed a good approximation for the convective-reactive i-process sites.

### The high-mass endpoint and the back reaction $\mathsf{^{209}Bi}$  to $\mathsf{^{56}Fe}$
On the high end no n-captures are allowed for isobars with atomic mass $\mathsf{A\geq209}$. In that way we force all flux to end in $\mathsf{^{209}Bi}$.

This neutron-capture reaction will need to conserve mass and turn one $\mathsf{^{209}Bi}$ into one $\mathsf{^{56}Fe}$ for each $\mathsf{^{56}Fe}$ that captures a neutron. This is achieved by setting the reaction rate  $\mathsf{<\sigma v>_{Bi209+n}}$ according to the condition $\mathsf{N_{Fe56} <\sigma v>_{Fe56+n} = N_{Bi209} <\mathsf{\sigma v>_{Bi209+n}}}$ [(Clayton 1968, Jorissen & Arnould 1989)](ref).

With this reaction rate the back reaction is

```
    Bi209  + n  -> Fe56 + 154L1
```

## The resulting reaction network
### Artifical species
The articifial species are already defined in `ppn_physics.input`

```
  40 L   1   1.  0. F  1.000d+99 s
  41 G   1   1.  0. F  1.000d+99 s
```

### Reactions
These reactions  need to be added to the sandbox `ppn_physics.input`.
```
 111 T  1 G   1   1 L   1  0 NEUT    1 G   1     0.000   VITAL  (v,v)  99   1.000E+00
 112 T  1 BI209   0 NEUT   9 L   1   1 FE 56     0.000   VITAL  (v,v)  99   1.000E+00
```

#### Notes
1. The reaction rate numbers 111 and 112 are place holders.
2. The `ppn_physics.input` format only allows one character for the number of output particles. It is set to `9 L   1` but should be set to `154 L   1`. In the code there must be made arrangements that when this reaction is read the corresponding parameter is set to 154 instead of 9.


### Initial abundance
The initial abundances that need to be specified are only those of $\mathsf{^1G}$, $\mathsf{^1L}$ and $\mathsf{^{56}Fe}$. 

$\mathsf{^1G}$ will stay constant by definition. It therefore does not matter to which value it is set. But the value should be large enough that within iterations corrections are not larger than the abundance of $\mathsf{^1G}$. It's molar abundance should therefore be a few times that $\mathsf{^{56}Fe}$. 

The exact amount of $\mathsf{^{56}Fe}$ initially does not matter either. It must be choosen smaller than $\mathsf{^1L}$, keeping in mind that we specify mass fractions but the relations apply to molar fractions. Thus, any initial choice that satisfies $\mathsf{Y(^1L) \gt Y(^1G)  \gt Y(^{56}Fe})$ should work, and the equilibrium abundance ratios should not depend on initial conditions.

One could certainly figure out more accurately what these numbers should be. But one may start with
```
Fe56     0.01
G1       0.05
L1       0.94
```
And everything else set to `0`.

## Expected operation of the network
By definition the abundance of $\mathsf{^1G}$ is constant.  This minimum can be estimated from the adopted molar abundance $\mathsf{^{56}Fe}$. In the solar abundance distribution it is $\mathsf{2\times10^{-5}}$. However, the absolute $\mathsf{^{56}Fe}$ abundance is irrelevant in this equilibrium approach. In fact, it may be advisable to choose it substantially larger to mininmize round-off erros. Then, for the initial abundance of $\mathsf{^1G}$ any number larger by some factor, maybe 10, will do.

Initially the abundance of $\mathsf{^1L}$ will decrease as the `G1 + L1 -> n + G1` reaction produces neutrons at the rate demanded by the prescribed neutron density $\mathsf{N_n}$. No substantial amounts of $\mathsf{^1L}$ will be replenished until the reaction flow reaches $\mathsf{^{209}Bi}$. At that point the back reaction `Bi209  + n  -> Fe56 + 154L1` will replenish `L1` and the `L1` abundance will asymptotically approach a constant value `L1_final`, at which point the computation can be stopped and the equilibrium abundance has been established. 

As an aside, the difference `L1_initial - L1_final` is equal to the $\mathsf{n_{cap}}$, the number of neutrons captured per $\mathsf{^{56}Fe}$ and normally represents the neutron exposure. In this context the neutron exposure is meaningless.  

For higher $\mathsf{N_n}$ `L1_initial - L1_final`  will be larger because the reaction flow will go further away from stability where generally speaking n-capture cross sections are low than closer to stability. This means that more material will pile up in this giant bottleneck compared to the more efficient pathway through the valley of stability. 



## Alternative option: solve equilibrium equations
What is described above is of course the time evolution towards an equilibrium solution. This could also be found by just solving the linear set of equations that derive from setting the left-hand side of all network equations above to zero, while ommitting equations for neutrons and setting $\mathsf{N_n}$ to prescribed constant values. The back reaction and the artificial neutron source need to be included, of course. Technically this can be easily done by letting the `ppn` code write a `networksetup.txt` file for input T and $\mathsf{\rho}$ values, so that the rates for those conditions are written out. All unwanted reactions and species are set to `F`. One can then use the `networksetup.txt`  file as input, and solve in a separate Python program the linear set of equations. The result is an equilibrium abundance distribution. This may not be saving implementation time though. 

## Implementation plan
1. Add the reactions above to sandbox and set the rates in vital initially to 0. Make sure all other infrastructure for the additional sandbox rates is taken care of.
2. Make sure number of L1 particles when reading reaction 112 is changed from 9 to 154 according to note 2 above; escape by a master switch. The switch can be a positive or negative neutron number density specified in `ppn_frame.input`.
3. In `vital` set rates for reactions 111 and 112 to 0. Then call `rnetw...`. Then pass v and yps arrays into a new subroutine `vital_nconsteq` and calculate and return v(111) and v(112). Instruction for both reactions 111 and 112 are given above. .
3. Generate an initial abundance file using the `nugridpy.utils.iniabu` method. 
4. Read in prescribed constant neutron density and use to calculate reaction rate of neutron source reaction. Note that this neutron density is not the abundance of neutrons calculated in the code!
4. Test and debug.
5. Add a stopping condition `delta_1L/dt < min_delta1Ldt` where `min_delta1Ldt` is an input parameter.

## Application plan
Once completed the constant $\mathsf{N_n}$  equilibrium network will be used to 
* Establish the equilibrium neutron capture abundance distribution as a function of $\mathsf{N_n}$. The range of $\mathsf{N_n}$ will cover the whole range from s process through i, n and up to r process.
* For each $\mathsf{N_n}$ an MC simulation will determine nuclear physics error bars.
* Plots will be made of abundance ratios of neighboring elements, including one-over neighbors. 
* A correlation analysis will determine how broad _neighboring_ is to be applied, i.e. how far away can elements be in mass so that their ratios correlate with $\mathsf{N_n}$?
* A correlation analysis will also determine which elemental ratios are most sensitive to $\mathsf{N_n}$, and we expect this to vary with $\mathsf{N_n}$ regime.
* Finally, the equilibrium abundance distributions will be fitted against libraries of observed heavy element abundance distributions in a variety of stars. For each star a histogram of neutron densities having contributed will be derived, and - using the observational and simulation errors - be translated into a maximum likelyhood estimate, possibly using an MCMC approach.